In [12]:
import string
printable = set(string.printable)
printable.remove("\n")
printable.remove("\t")
printable.remove("\r")
import json 

from nltk.tokenize import word_tokenize

In [30]:
def get_qid2query(year):
    qid2query = {}
    f = open("src/main/resources/topics-and-qrels/topics.microblog{}.txt".format(year))
    query_tag = "query" if year != 2011 else "title"
    for l in f:
        ind = l.find("Number: MB")
        if ind >= 0:
            ind2 = l.find("</num>")
            qid = l[ind+10:ind2-1]
            qid = int(qid) 
        ind = l.find("<{}>".format(query_tag))
        if ind >= 0:
            ind2 = l.find("</{}>".format(query_tag))
            query = l[ind+8:ind2-1]
            qid2query[qid] = query
    return qid2query

def getdoc(docid, folder):
    f = open("{}/{}".format(folder, docid))
    doc = json.load(f)
#     if doc['in_reply_to_user_id']:
#         print(doc['in_reply_to_user_id'])
#         return None
    return doc['text']

def get_qid2reldocids(year):
    f = open("src/main/resources/topics-and-qrels/qrels.microblog{}.txt".format(year))
    qid2reldocids = {}
    qid2reldocids[50] = set()
    for l in f:
        qid, _, docid, score = l.replace("\n", "").strip().split()
        qid = int(qid)
        if score != "0" and score != "-2":
            if qid not in qid2reldocids:
                qid2reldocids[qid] = set()
            qid2reldocids[qid].add(docid)
    return qid2reldocids

def get_doc_from_index(indexes, docid):
    doc_raw = indexes.getRawDocument(JString(docid))
    doc = json.loads(doc_raw)['text']
    return doc

In [202]:
docid = "34951007502995456"
folder = "run.mb11.bm25.topics.microblog2011.docids.txt_rawdocs.dump"
folder = "qrels.microblog2011.docids.txt_rawdocs.dump"
f = open("{}/{}".format(folder, docid))
doc = json.load(f)
doc

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Feb 08 12:25:44 +0000 2011',
 'favorited': False,
 'geo': None,
 'id': 34951007502995456,
 'id_str': '34951007502995456',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'place': None,
 'retweet_count': 1,
 'retweeted': False,
 'source': '<a href="http://www.ubertwitter.com/bb/download.php" rel="nofollow">ÜberSocialOrig</a>',
 'text': "I'd very much appreciate it if people would stop broadcasting asking me to add people on BBM.",
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Mon Apr 06 16:39:31 +0000 2009',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'enter information here that hardly anyonr will read.',
  'favourites_count': 911,
  'follow_request_sent': None,
  'followers_count': 1507,
  'following': None,
  'friends_count': 1050,
  'g

In [20]:
def write2file(out, sim, qid2query, doc_folder, qid, docid, remove_url, tokenize):
    a = qid2query[qid]
    try:
        b = getdoc(docid, doc_folder)
    except Exception as e:
        return False
    if b is None:
        print("empty doc")
        return False
    # remove characters that cannot be encoded by ASCII
    b = "".join(filter(lambda x: x in printable, b))
    if remove_url:
        bs = []
        for w in b.split():
            if not w.startswith("http"):
                bs.append(w)
        b = " ".join(bs)
    if tokenize:
        b = ' '.join(word_tokenize(b))
    a = a.lower()
    b = b.lower()
    out.write("{}\t{}\t{}\t{}\t{}\n".format(sim, a, b, qid, docid))
    out.flush()
    return True

def gen_test_data(f_input, f_output, doc_folder, remove_url=True, tokenize=True):
    count = 0 
    with open(f_output, "w") as out, open(f_input) as f:
        for l in f:
            qid, _, docid, _, score, _ = l.split()
            qid = int(qid)
            if qid not in qid2reldocids:
                print("{} not in qid2reldocids".format(qid))
                continue
            sim = 1 if docid in qid2reldocids[qid] else 0
            if not write2file(out, sim, qid2query, doc_folder, qid, docid, remove_url, tokenize):
                count += 1
    print("error doc: {}".format(count))
                
def gen_qrel_data(f_input, f_output, doc_folder, remove_url=True, tokenize=True):
    count = 0 
    with open(f_output, "w") as out, open(f_input) as f:
        for l in f:
            qid, _, docid, sim = l.replace("\n", "").split()
            qid = int(qid)
            if sim == "2":
                sim = "1"
            if sim == "-2":
                sim = "0"
            assert sim in ["0", "1"]
            if not write2file(out, sim, qid2query, doc_folder, qid, docid, remove_url, tokenize):
                count += 1
    print("error doc: {}".format(count))
    

```shell
python run2docid.py # modify the path variable first
sh target/appassembler/bin/IndexUtils \
-index /tuna1/indexes/lucene-index.mb11.pos+docvectors+rawdocs \
-dumpRawDocs run.mb11.ql.topics.microblog2011.docids.txt
```

In [17]:
%%time
# generate test data
for year in range(2011, 2015):
    year2index = {2011: "mb11", 2012: "mb11", 2013: "mb13", 2014: "mb13"}
    qid2query = get_qid2query(year)
    qid2reldocids = get_qid2reldocids(year)
    f_input = "run.{}.ql.topics.microblog{}.txt".format(year2index[year], year)
    doc_folder = "run.{}.ql.topics.microblog{}.docids.txt_rawdocs.dump".format(year2index[year], year)
    f_output = "src/main/python/rerank/MatchZoo/data/tweets/TweetCorpus/test_ql_{}.txt".format(year)
    gen_test_data(f_input, f_output, doc_folder, tokenize=False, remove_url=False)

50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2reldocids
50 not in qid2re

76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2reldocids
76 not in qid2re

CPU times: user 49 s, sys: 26.4 s, total: 1min 15s
Wall time: 4min 53s


## Test and evaluation
```shell
cd src/main/python/rerank/MatchZoo/data/tweets
sh ./run_data.sh
cd ../../
python matchzoo/main.py --phase predict --model_file ./examples/tweets/config/duet_tweet.config
cd ../../../../../
./eval/trec_eval.9.0/trec_eval src/main/resources/topics-and-qrels/qrels.microblog2014.txt src/main/python/rerank/MatchZoo/data/tweets/predict.test.txt
```    

In [201]:
# try IndexUtil
year = 2014
f = open("src/main/resources/topics-and-qrels/qrels.microblog{}.txt".format(year))
with open("qrels.microblog{}.docids.txt".format(year), "w") as out:
    for l in f:
        qid, _, docid, sim = l.replace("\n", "").split()
        out.write("{}\n".format(docid))

```shell
sh target/appassembler/bin/IndexUtils \
-index /tuna1/indexes/lucene-index.mb13.pos+docvectors+rawdocs \
-dumpRawDocs qrels.microblog2013.docids.txt
```

In [21]:
%%time
# generate data from qrels file
for year in range(2011, 2015):
    qid2query = get_qid2query(year)
    qid2reldocids = get_qid2reldocids(year)
    f_input = "src/main/resources/topics-and-qrels/qrels.microblog{}.txt".format(year)
    f_output = "src/main/python/rerank/MatchZoo/data/tweets/TweetCorpus/train_qrels_{}.txt".format(year)
    doc_folder = "qrels.microblog{}.docids.txt_rawdocs.dump".format(year)
    gen_qrel_data(f_input, f_output, doc_folder, tokenize=False, remove_url=False)


error doc: 8055
error doc: 0
error doc: 1
error doc: 0
CPU times: user 58.8 s, sys: 32.2 s, total: 1min 30s
Wall time: 5min 53s


```shell
cd src/main/python/rerank/MatchZoo/data/tweets/TweetCorpus/
cat train_qrels_2011.txt train_qrels_2012.txt train_qrels_2013.txt > train_qrels_2011_2012_2013_all.txt
python split_train_dev.py
```

## Train
```shell
cd src/main/python/rerank/MatchZoo/data/tweets
sh ./run_data.sh
cd ../../
python matchzoo/main.py --phase train --model_file ./examples/tweets/config/duet_tweet.config
```


In [22]:
qlen = []
for qid in qid2query:
    qlen.append(len(qid2query[qid].split()))

In [28]:
doclen = []
f2 = open("src/main/python/rerank/MatchZoo/data/tweets/TweetCorpus/train_qrels_2014.txt")
for l in f2:
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    doclen.append(len(b.split()))

In [29]:
max(doclen)

49

In [87]:
# some issue with jnius
